# CEI Score Impact on Stock Excess Returns Analysis (Interactive)

This notebook analyzes how Corporate Equality Index (CEI) scores affect stock **excess returns** around release dates with interactive visualizations.

## Key Changes:
- **Interactive plots** using Plotly
- **Proper bin ordering** with 100-100 as highest bin
- **Excess returns** instead of raw returns (stock return - market return)

## Analysis Steps:
1. Load CEI scores and stock price data
2. Calculate excess returns using market benchmark
3. Aggregate companies by CEI score bins (properly ordered)
4. Analyze excess returns by year and score bin around release dates
5. Create interactive analysis across all years

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up Plotly for Jupyter
pio.renderers.default = "notebook"

# Color palette for consistent visualization
color_palette = px.colors.qualitative.Set3

## 1. Load and Prepare Data

In [ ]:
# Load CEI data
print("Loading CEI data...")
cei_df = pd.read_csv('../data/processed/cei_with_dates.csv')
print(f"CEI records: {len(cei_df):,}")
print(f"Years covered: {sorted(cei_df['year'].unique())}")

# Load stock price data
print("\nLoading stock price data...")
stock_df = pd.read_csv('../data/processed/stock_prices_event_window.csv')
stock_df['date'] = pd.to_datetime(stock_df['date'])
stock_df['cei_release_date'] = pd.to_datetime(stock_df['cei_release_date'])
print(f"Stock price records: {len(stock_df):,}")
print(f"Unique firms: {stock_df['cusip6'].nunique()}")
print(f"Date range: {stock_df['date'].min()} to {stock_df['date'].max()}")

# Display data info
print("\nCEI Data Sample:")
display(cei_df.head())

print("\nStock Data Sample:")
display(stock_df.head())

In [ ]:
# Create CUSIP6 from CEI data for matching
def get_cusip6(cusip):
    """Extract first 6 digits of CUSIP."""
    if pd.isna(cusip):
        return None
    cusip_str = str(cusip).strip()
    if len(cusip_str) >= 6:
        return cusip_str[:6]
    return None

cei_df['cusip6'] = cei_df['cusip'].apply(get_cusip6)
cei_df = cei_df.dropna(subset=['cusip6', 'cei_score'])

print(f"CEI records with valid CUSIP6 and scores: {len(cei_df):,}")

In [ ]:
# Create CEI score bins with proper ordering (100-100 as highest)
def create_score_bin_ordered(score):
    """Create score bins with proper ordering: 0-9, 10-19, ..., 90-99, 100-100"""
    if pd.isna(score):
        return None
    
    if score == 100:
        return "100-100"  # Perfect score gets its own bin
    else:
        bin_start = int(score // 10) * 10
        bin_end = bin_start + 9
        return f"{bin_start}-{bin_end}"

cei_df['score_bin'] = cei_df['cei_score'].apply(create_score_bin_ordered)

# Define proper bin order
bin_order = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99', '100-100']

# Show score distribution
print("CEI Score Distribution by Bin (Properly Ordered):")
score_dist = cei_df['score_bin'].value_counts().reindex(bin_order, fill_value=0)
print(score_dist)

# Interactive bar plot of score distribution
fig = go.Figure(data=[
    go.Bar(x=score_dist.index, y=score_dist.values, 
           marker_color='lightblue', 
           text=score_dist.values,
           textposition='auto')
])

fig.update_layout(
    title='Distribution of CEI Scores by Bin',
    xaxis_title='CEI Score Bin',
    yaxis_title='Number of Company-Year Observations',
    showlegend=False,
    height=500
)

fig.show()

## 2. Calculate Excess Returns

In [ ]:
# Calculate excess returns (stock return - market return)
# Use value-weighted market return (vwretd) as benchmark
print("Calculating excess returns...")

# Check if market return columns exist
market_cols = ['vwretd', 'vwretx', 'ewretd', 'ewretx']
available_market_cols = [col for col in market_cols if col in stock_df.columns]
print(f"Available market return columns: {available_market_cols}")

if 'vwretd' in stock_df.columns:
    # Use value-weighted return with dividends as benchmark
    stock_df['excess_return'] = stock_df['RET'] - stock_df['vwretd']
    benchmark_used = 'vwretd (Value-Weighted Market Return with Dividends)'
elif 'vwretx' in stock_df.columns:
    # Use value-weighted return without dividends
    stock_df['excess_return'] = stock_df['RET'] - stock_df['vwretx']
    benchmark_used = 'vwretx (Value-Weighted Market Return without Dividends)'
elif 'ewretd' in stock_df.columns:
    # Use equal-weighted return with dividends
    stock_df['excess_return'] = stock_df['RET'] - stock_df['ewretd']
    benchmark_used = 'ewretd (Equal-Weighted Market Return with Dividends)'
else:
    # Fallback: use a simple estimate (assume 0.03% daily market return)
    stock_df['excess_return'] = stock_df['RET'] - 0.0003
    benchmark_used = 'Simple estimate (0.03% daily market return)'
    
print(f"Benchmark used for excess returns: {benchmark_used}")
print(f"Mean excess return: {stock_df['excess_return'].mean()*100:.4f}%")
print(f"Std excess return: {stock_df['excess_return'].std()*100:.4f}%")

## 3. Merge CEI and Stock Data

In [ ]:
# Merge CEI data with stock data
# First, create year from release date in stock data
stock_df['cei_year'] = stock_df['cei_release_date'].dt.year

# Merge on cusip6 and year
merged_df = stock_df.merge(
    cei_df[['cusip6', 'year', 'cei_score', 'score_bin', 'employer']], 
    left_on=['cusip6', 'cei_year'], 
    right_on=['cusip6', 'year'], 
    how='inner'
)

print(f"Merged records: {len(merged_df):,}")
print(f"Unique firms in merged data: {merged_df['cusip6'].nunique()}")
print(f"Years with data: {sorted(merged_df['year'].unique())}")

# Clean data - remove missing returns
merged_df = merged_df.dropna(subset=['RET', 'excess_return'])
print(f"Records with valid returns: {len(merged_df):,}")

display(merged_df[['cusip6', 'date', 'cei_score', 'score_bin', 'RET', 'excess_return', 'days_from_release']].head())

## 4. Analyze Excess Returns by Year and Score Bin

In [ ]:
# Calculate average excess returns by year, score bin, and days from release
yearly_analysis = merged_df.groupby(['year', 'score_bin', 'days_from_release'])['excess_return'].agg([
    'mean', 'std', 'count'
]).reset_index()

yearly_analysis.columns = ['year', 'score_bin', 'days_from_release', 'avg_excess_return', 'std_excess_return', 'count']

print("Sample of yearly analysis:")
display(yearly_analysis.head(10))

In [ ]:
# Interactive plot of excess returns by score bin for each year around release dates
years_to_plot = sorted(merged_df['year'].unique())[:6]  # Plot first 6 years

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f'Excess Returns Around CEI Release - {year}' for year in years_to_plot],
    vertical_spacing=0.12,
    horizontal_spacing=0.08
)

# Color mapping for consistent colors across score bins
available_bins = sorted(yearly_analysis['score_bin'].unique(), key=lambda x: bin_order.index(x) if x in bin_order else 999)
color_map = {bin_name: color_palette[i % len(color_palette)] for i, bin_name in enumerate(available_bins)}

for i, year in enumerate(years_to_plot):
    row = i // 3 + 1
    col = i % 3 + 1
    
    year_data = yearly_analysis[yearly_analysis['year'] == year]
    
    # Plot each score bin
    for score_bin in available_bins:
        bin_data = year_data[year_data['score_bin'] == score_bin]
        if not bin_data.empty:
            fig.add_trace(
                go.Scatter(
                    x=bin_data['days_from_release'], 
                    y=bin_data['avg_excess_return'] * 100,
                    mode='lines+markers',
                    name=f'CEI {score_bin}',
                    line=dict(color=color_map[score_bin]),
                    showlegend=(i == 0),  # Only show legend for first subplot
                    hovertemplate=f'CEI {score_bin}<br>Day: %{{x}}<br>Excess Return: %{{y:.3f}}%<extra></extra>'
                ),
                row=row, col=col
            )
    
    # Add release date line
    fig.add_vline(x=0, line_dash="dash", line_color="red", opacity=0.7, row=row, col=col)

fig.update_layout(
    height=800,
    title_text="Excess Returns Around CEI Release Dates by Year and Score Bin",
    title_x=0.5
)

fig.update_xaxes(title_text="Days from Release")
fig.update_yaxes(title_text="Average Excess Return (%)")

fig.show()

## 5. Aggregate Analysis Across All Years

In [ ]:
# Aggregate across all years
overall_analysis = merged_df.groupby(['score_bin', 'days_from_release'])['excess_return'].agg([
    'mean', 'std', 'count'
]).reset_index()

overall_analysis.columns = ['score_bin', 'days_from_release', 'avg_excess_return', 'std_excess_return', 'count']

# Calculate standard error
overall_analysis['std_error'] = overall_analysis['std_excess_return'] / np.sqrt(overall_analysis['count'])

print("Overall analysis sample:")
display(overall_analysis.head(10))

In [ ]:
# Interactive main plot: Excess Returns vs Days from Release by Score Bin
fig = go.Figure()

# Sort bins according to our defined order
available_bins = sorted(overall_analysis['score_bin'].unique(), key=lambda x: bin_order.index(x) if x in bin_order else 999)

for score_bin in available_bins:
    bin_data = overall_analysis[overall_analysis['score_bin'] == score_bin].sort_values('days_from_release')
    
    if not bin_data.empty:
        # Main line
        fig.add_trace(
            go.Scatter(
                x=bin_data['days_from_release'], 
                y=bin_data['avg_excess_return'] * 100,
                mode='lines+markers',
                name=f'CEI {score_bin}',
                line=dict(width=3),
                marker=dict(size=8),
                hovertemplate=f'CEI {score_bin}<br>Day: %{{x}}<br>Excess Return: %{{y:.4f}}%<br>Count: {bin_data["count"].iloc[0]}<extra></extra>'
            )
        )
        
        # Confidence intervals
        upper_bound = (bin_data['avg_excess_return'] + 1.96 * bin_data['std_error']) * 100
        lower_bound = (bin_data['avg_excess_return'] - 1.96 * bin_data['std_error']) * 100
        
        fig.add_trace(
            go.Scatter(
                x=bin_data['days_from_release'].tolist() + bin_data['days_from_release'].tolist()[::-1],
                y=upper_bound.tolist() + lower_bound.tolist()[::-1],
                fill='toself',
                fillcolor=fig.data[-1].line.color,
                opacity=0.2,
                line=dict(width=0),
                name=f'CEI {score_bin} CI',
                showlegend=False,
                hoverinfo='skip'
            )
        )

# Add vertical line at release date
fig.add_vline(x=0, line_dash="dash", line_color="red", line_width=3, opacity=0.8)
fig.add_hline(y=0, line_color="black", opacity=0.5)

fig.update_layout(
    title={
        'text': 'Stock Excess Returns Around CEI Release Dates by Score (All Years Combined)',
        'x': 0.5,
        'font': {'size': 18}
    },
    xaxis_title='Days from CEI Release Date',
    yaxis_title='Average Daily Excess Return (%)',
    height=600,
    hovermode='x unified',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.01
    )
)

# Add annotation for release date
fig.add_annotation(
    x=0, y=fig.layout.yaxis.range[1] if hasattr(fig.layout.yaxis, 'range') else 0.5,
    text="CEI Release Date",
    showarrow=True,
    arrowhead=2,
    arrowcolor="red",
    ax=20,
    ay=-30
)

fig.show()

In [ ]:
# Interactive cumulative excess returns plot
fig = go.Figure()

for score_bin in available_bins:
    bin_data = overall_analysis[overall_analysis['score_bin'] == score_bin].sort_values('days_from_release')
    
    if not bin_data.empty:
        # Calculate cumulative excess returns
        cumulative_returns = (1 + bin_data['avg_excess_return']).cumprod() - 1
        
        fig.add_trace(
            go.Scatter(
                x=bin_data['days_from_release'], 
                y=cumulative_returns * 100,
                mode='lines+markers',
                name=f'CEI {score_bin}',
                line=dict(width=3),
                marker=dict(size=8),
                hovertemplate=f'CEI {score_bin}<br>Day: %{{x}}<br>Cumulative Excess Return: %{{y:.4f}}%<extra></extra>'
            )
        )

fig.add_vline(x=0, line_dash="dash", line_color="red", line_width=3, opacity=0.8)
fig.add_hline(y=0, line_color="black", opacity=0.5)

fig.update_layout(
    title={
        'text': 'Cumulative Excess Returns Around CEI Release Dates by Score',
        'x': 0.5,
        'font': {'size': 18}
    },
    xaxis_title='Days from CEI Release Date',
    yaxis_title='Cumulative Excess Return (%)',
    height=600,
    hovermode='x unified',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.01
    )
)

fig.show()

## 6. Statistical Analysis

In [ ]:
# Calculate event window excess returns (e.g., 3 days before to 3 days after)
event_window = merged_df[merged_df['days_from_release'].between(-3, 3)]

# Calculate cumulative excess returns for each firm during event window
firm_event_returns = event_window.groupby(['cusip6', 'year', 'score_bin'])['excess_return'].sum().reset_index()
firm_event_returns.columns = ['cusip6', 'year', 'score_bin', 'event_excess_return']

print("Event Window Excess Returns Summary:")
summary_stats = firm_event_returns.groupby('score_bin')['event_excess_return'].agg([
    'count', 'mean', 'std', 'min', 'max'
])
summary_stats.columns = ['N_firms', 'Mean_Excess_Return', 'Std_Excess_Return', 'Min_Excess_Return', 'Max_Excess_Return']
summary_stats['Mean_Excess_Return_pct'] = summary_stats['Mean_Excess_Return'] * 100

# Reorder by our bin order
summary_stats = summary_stats.reindex([bin for bin in bin_order if bin in summary_stats.index])
display(summary_stats)

In [ ]:
# Interactive box plot of event excess returns by score bin
fig = go.Figure()

# Sort bins according to our defined order
available_event_bins = [bin for bin in bin_order if bin in firm_event_returns['score_bin'].unique()]

for score_bin in available_event_bins:
    returns = firm_event_returns[firm_event_returns['score_bin'] == score_bin]['event_excess_return'] * 100
    
    fig.add_trace(
        go.Box(
            y=returns,
            name=f'CEI {score_bin}',
            boxpoints='outliers',
            hovertemplate=f'CEI {score_bin}<br>Excess Return: %{{y:.3f}}%<extra></extra>'
        )
    )

fig.update_layout(
    title={
        'text': 'Distribution of 7-Day Event Window Excess Returns by CEI Score',
        'x': 0.5,
        'font': {'size': 16}
    },
    xaxis_title='CEI Score Bin',
    yaxis_title='Event Window Excess Return (%)',
    height=600,
    showlegend=False
)

fig.show()

In [ ]:
# Test for significant differences between high and low CEI scores
from scipy import stats

# Compare highest scoring companies (100-100 and 90-99) vs lowest (0-19)
high_scores = firm_event_returns[firm_event_returns['score_bin'].isin(['100-100', '90-99'])]['event_excess_return']
low_scores = firm_event_returns[firm_event_returns['score_bin'].isin(['0-9', '10-19'])]['event_excess_return']

if len(high_scores) > 0 and len(low_scores) > 0:
    t_stat, p_value = stats.ttest_ind(high_scores, low_scores)
    
    print(f"Statistical Test: High CEI (90-100) vs Low CEI (0-19) Scores")
    print(f"High CEI mean excess return: {high_scores.mean()*100:.3f}%")
    print(f"Low CEI mean excess return: {low_scores.mean()*100:.3f}%")
    print(f"Difference: {(high_scores.mean() - low_scores.mean())*100:.3f} percentage points")
    print(f"T-statistic: {t_stat:.3f}")
    print(f"P-value: {p_value:.3f}")
    print(f"Significant at 5% level: {'Yes' if p_value < 0.05 else 'No'}")
    
    # Additional test: Perfect score (100-100) vs others
    if '100-100' in firm_event_returns['score_bin'].unique():
        perfect_scores = firm_event_returns[firm_event_returns['score_bin'] == '100-100']['event_excess_return']
        other_scores = firm_event_returns[firm_event_returns['score_bin'] != '100-100']['event_excess_return']
        
        if len(perfect_scores) > 1:
            t_stat_perfect, p_value_perfect = stats.ttest_ind(perfect_scores, other_scores)
            print(f"\nAdditional Test: Perfect CEI (100-100) vs Others")
            print(f"Perfect CEI mean excess return: {perfect_scores.mean()*100:.3f}%")
            print(f"Others mean excess return: {other_scores.mean()*100:.3f}%")
            print(f"Difference: {(perfect_scores.mean() - other_scores.mean())*100:.3f} percentage points")
            print(f"T-statistic: {t_stat_perfect:.3f}")
            print(f"P-value: {p_value_perfect:.3f}")
            print(f"Significant at 5% level: {'Yes' if p_value_perfect < 0.05 else 'No'}")
            
else:
    print("Insufficient data for statistical test")

## 7. Interactive Heatmap Analysis

In [ ]:
# Create interactive heatmap of excess returns by score bin and days from release
pivot_data = overall_analysis.pivot(index='score_bin', columns='days_from_release', values='avg_excess_return')

# Reorder rows according to our bin order
pivot_data = pivot_data.reindex([bin for bin in bin_order if bin in pivot_data.index])

fig = go.Figure(data=go.Heatmap(
    z=pivot_data.values * 100,  # Convert to percentage
    x=pivot_data.columns,
    y=pivot_data.index,
    colorscale='RdBu',
    zmid=0,  # Center colorscale at 0
    hoverinfotemplate='CEI Score: %{y}<br>Days from Release: %{x}<br>Avg Excess Return: %{z:.4f}%<extra></extra>',
    colorbar=dict(title="Avg Excess Return (%)")
))

fig.update_layout(
    title={
        'text': 'Heatmap: Average Excess Returns by CEI Score and Days from Release',
        'x': 0.5,
        'font': {'size': 16}
    },
    xaxis_title='Days from CEI Release Date',
    yaxis_title='CEI Score Bin',
    height=600
)

# Add vertical line at release date
fig.add_vline(x=0, line_dash="dash", line_color="white", line_width=3)

fig.show()

## 8. Key Findings Summary

In [ ]:
# Summary statistics table
print("=== CEI STOCK EXCESS RETURN ANALYSIS SUMMARY ===")
print(f"\nData Coverage:")
print(f"  • Total observations: {len(merged_df):,}")
print(f"  • Unique firms: {merged_df['cusip6'].nunique()}")
print(f"  • Years analyzed: {len(merged_df['year'].unique())}")
print(f"  • CEI release dates: {len(merged_df['cei_release_date'].unique())}")
print(f"  • Benchmark used: {benchmark_used}")

print(f"\nScore Distribution (Properly Ordered):")
for score_bin in [bin for bin in bin_order if bin in summary_stats.index]:
    n_firms = summary_stats.loc[score_bin, 'N_firms']
    mean_ret = summary_stats.loc[score_bin, 'Mean_Excess_Return_pct']
    print(f"  • CEI {score_bin}: {n_firms} firms, {mean_ret:.3f}% avg excess return")

print(f"\nEvent Window Analysis (-3 to +3 days):")
if len(high_scores) > 0 and len(low_scores) > 0:
    print(f"  • High CEI firms (90-100): {high_scores.mean()*100:.3f}% average excess return")
    print(f"  • Low CEI firms (0-19): {low_scores.mean()*100:.3f}% average excess return")
    print(f"  • Difference: {(high_scores.mean() - low_scores.mean())*100:.3f} percentage points")
    print(f"  • Statistical significance: {'Yes' if p_value < 0.05 else 'No'} (p = {p_value:.3f})")
    
    if '100-100' in firm_event_returns['score_bin'].unique():
        perfect_scores = firm_event_returns[firm_event_returns['score_bin'] == '100-100']['event_excess_return']
        print(f"  • Perfect CEI firms (100-100): {perfect_scores.mean()*100:.3f}% average excess return")
        print(f"  • Perfect vs Others significance: {'Yes' if 'p_value_perfect' in locals() and p_value_perfect < 0.05 else 'No'}")

print("\n" + "="*60)
print("\nKey Improvements in this Interactive Version:")
print("  ✓ Interactive Plotly visualizations with hover details")
print("  ✓ Proper CEI score bin ordering (100-100 as highest)")
print("  ✓ Excess returns analysis (market-adjusted)")
print("  ✓ Enhanced statistical testing")
print("  ✓ Interactive heatmap visualization")
print("  ✓ Confidence intervals on main plots")